## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)

> **DBA Procedures**
> 
> - Adam Machanic who isactive procedure download (Q0)
> - Details of index statistics, seeks, scans etc (Q1)
> - Details of missing index statistics, seeks, scans etc (Q2)
> - Find blocking information (Q3)
> - Find details of procedure stats (Q4)
> - Find details of Server space (Q5)

> - Adam Machanic must have script  (Q0)
[sp\_whoisactive](http://whoisactive.com/downloads/)

> **DBA index statistics**
> - Details of index statistics, seeks, scans etc(Query 1)
 

In [ ]:
-- DBA_IndexStats.sql  #SQL
      
SELECT      
	db_name() AS DatabaseName, 
	o.name,      
	indexname=i.name,    
	i.type_desc, 
	i.index_id,      
	user_seeks,   
	user_scans,
	user_lookups,
	reads=user_seeks + user_scans + user_lookups,         
	writes =  user_updates,         
	rows = (SELECT SUM(p.rows) FROM sys.partitions p WHERE p.index_id = s.index_id AND s.object_id = p.object_id),      
	8 * (a.used_pages) AS 'Indexsize(KB)',  
	CASE      
		WHEN s.user_updates < 1 THEN 100      
		ELSE 1.00 * (s.user_seeks + s.user_scans + s.user_lookups) / s.user_updates      
	END AS reads_per_write,      
	'DROP INDEX ' + QUOTENAME(i.name)+' ON ' + QUOTENAME(c.name) + '.' + QUOTENAME(OBJECT_NAME(s.object_id)) as 'drop statement'      
FROM 
	sys.dm_db_index_usage_stats s        
INNER JOIN       
	sys.indexes i ON i.index_id = s.index_id AND s.object_id = i.object_id         
INNER JOIN       
	sys.objects o on s.object_id = o.object_id      
INNER JOIN       
	sys.schemas c on o.schema_id = c.schema_id  
INNER JOIN    
	 sys.partitions AS p ON p.OBJECT_ID = i.OBJECT_ID AND p.index_id = i.index_id  
INNER JOIN   
	 sys.allocation_units AS a ON a.container_id = p.partition_id  
WHERE       
		(OBJECTPROPERTY(s.object_id,'IsUserTable') = 1  or OBJECTPROPERTY(s.object_id,'IsView') = 1)
	 AND       
		s.database_id = DB_ID() 
	 AND       
		i.name IS NOT NULL
	AND      
		i.is_unique_constraint = 0 
  ORDER BY O.name desc      


> ### DBA Missing indexes
> - Details of missing index statistics, seeks, scans etc(Query 2)
> - Will run in context of current database need to update use statement

In [ ]:
-- DBA_MissingIndexes.sql  #SQL
USE Master
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
SET NOCOUNT ON


DECLARE
 @TableName sysname = NULL,
 @SchemeName sysname =null,
 @Sort Tinyint =1,
 @IncludeExcluded BIT = NULL,
 @CountOnly BIT = NULL
  
-----------------------------------------------------------------  
-- Object:   sp_DBA_MissingIndexes  
-- Written By:  
-- Date Written:   
-- Purpose:   Return missing index information
-- Usage:     
--    sp_DBA_MissingIndexes  @TableName='dbCodelookup'
-- Calls:   N/A  
-----------------------------------------------------------------  

SELECT
  migs.avg_total_user_cost * (migs.avg_user_impact / 100.0) * (migs.user_seeks + migs.user_scans) AS improvement_measure,
  mid.statement AS TableName, 
  so.name,
  mid.object_id,
  	 replace(replace('CREATE NONCLUSTERED INDEX Ixn_' + object_name(mid.object_id) +'_' 
			  +  case 
					when mid.equality_columns is not null 
					and mid.inequality_columns is not null 
					then replace(mid.equality_columns,', ','_') + '_' + replace(mid.inequality_columns,', ','_')
					when mid.equality_columns is not null 
					and mid.inequality_columns is null 
					then replace(mid.equality_columns,', ','_')
					when mid.inequality_columns is not null 
					then replace(mid.inequality_columns,', ','_')
				  end
			 +         case 
			when mid.included_columns is not null 
			then '_I'
			else ''
			end
		   + ' on ' 
		   + ss.name COLLATE DATABASE_DEFAULT + '.'
			+ object_name(mid.object_id) + ' (' + 
			case 
					when mid.equality_columns is not null 
					and mid.inequality_columns is not null 
					then mid.equality_columns + ',' + mid.inequality_columns
					when mid.equality_columns is not null 
					and mid.inequality_columns is null 
					then mid.equality_columns
					when mid.inequality_columns is not null 
					then mid.inequality_columns
			end
			+ ')' + char(10)
			+ 
			case 
			when mid.included_columns is not null 
			then 'Include (' + mid.included_columns + ')'
			else ''
			end,'[',''),']','') as CreateIndexStmt,
			 equality_columns AS equality_columns,
			 inequality_columns AS inequality_columns,
			 included_columns AS included_columns,
			migs.unique_compiles,
			migs.user_seeks, 
			migs.user_scans,
			migs.avg_total_user_cost, 
			migs.avg_user_impact,
			migs.last_user_seek, 
			migs.last_user_scan,
			migs.system_seeks, 
			migs.system_scans,
			migs.avg_total_system_cost, 
			migs.avg_system_impact,
			migs.last_system_seek, 
			migs.last_system_scan,
			(CONVERT(Numeric(19,6), migs.user_seeks)+CONVERT(Numeric(19,6), migs.unique_compiles))*CONVERT(Numeric(19,6), migs.avg_total_user_cost)*CONVERT(Numeric(19,6), migs.avg_user_impact/100.0) AS Score
FROM sys.dm_db_missing_index_groups mig
INNER JOIN sys.dm_db_missing_index_group_stats migs ON migs.group_handle = mig.index_group_handle
INNER JOIN SYS.DM_DB_MISSING_INDEX_DETAILS mid ON mig.index_handle = mid.index_handle
LEFT JOIN sys.objects so on so.object_id=mid.object_id
LEFT JOIN sys.schemas ss ON ss.schema_id=so.schema_id
WHERE migs.avg_total_user_cost * (migs.avg_user_impact / 100.0) * (migs.user_seeks + migs.user_scans) > 10
AND so.name = COALESCE(@TableName,so.name) AND mid.database_id = db_id()
ORDER BY migs.avg_total_user_cost * migs.avg_user_impact * (migs.user_seeks + migs.user_scans) DESC



##  Missing indexes all databases

> **DBA Blocking chain**
> - Find blocking information (Query 3)
> - Shows lead blocker and chain of blocking

In [ ]:
-- DBA_BlockingChain.sql  #SQL
USE [master]
GO

SET ANSI_NULLS ON
GO

-----------------------------------------------------------------  
-- Object:   DBA_BlockingChain.sql  
-- Written By:  Martin Croft
-- Date Written:   
-- Purpose:   Shows lead blocking
-- Usage:     DBA_BlockingChain
-----------------------------------------------------------------  
-- Modified By  Modified Date  Description					Version 
-----------------------------------------------------------------  
-- {developer}  {date}    {description} {Version}  
-----------------------------------------------------------------  


IF (SELECT OBJECT_ID('TempDB.dbo.#T')) IS NOT NULL DROP TABLE #T
CREATE TABLE #T (spid INT, Blocked INT, BATCH VARCHAR(MAX))

INSERT INTO #T (Spid,blocked,Batch)
SELECT  spid ,
        blocked ,
        REPLACE(REPLACE(T.text, CHAR(10), ' '), CHAR(13), ' ') AS BATCH

FROM    sys.sysprocesses R
        CROSS APPLY sys.dm_exec_sql_text(R.sql_handle) T;

WITH    BLOCKERS ( SPID, BLOCKED, LEVEL, BATCH )
          AS ( SELECT   spid ,
                        blocked ,
                        CAST (REPLICATE('0', 4 - LEN(CAST (spid AS VARCHAR)))
                        + CAST (spid AS VARCHAR) AS VARCHAR(1000)) AS LEVEL ,
                        BATCH
               FROM     #T R
               WHERE    ( blocked = 0
                          OR blocked = spid
                        )
                        AND EXISTS ( SELECT *
                                     FROM   #T R2
                                     WHERE  R2.blocked = R.spid
                                            AND R2.blocked <> R2.spid )
               UNION ALL
               SELECT   R.spid ,
                        R.blocked ,
                        CAST (BLOCKERS.LEVEL
                        + RIGHT(CAST (( 1000 + R.spid ) AS VARCHAR(100)), 4) AS VARCHAR(1000)) AS LEVEL ,
                        R.BATCH
               FROM     #T AS R
                        INNER JOIN BLOCKERS ON R.blocked = BLOCKERS.SPID
               WHERE    R.blocked > 0
                        AND R.blocked <> R.spid
             )
    SELECT  N'    ' + REPLICATE(N'|         ', LEN(LEVEL) / 4 - 1)
            + CASE WHEN ( LEN(LEVEL) / 4 - 1 ) = 0 THEN 'HEAD -  '
                   ELSE '|------  '
              END + CAST (SPID AS NVARCHAR(10)) + N' ' + BATCH AS BLOCKING_TREE
    FROM    BLOCKERS
    ORDER BY LEVEL ASC;

GO




> **DBA Procedure statistics**
> - Find details of procedure stats (Query 4)
> - average, min and max times, cached time

In [ ]:
-- DBA_ProcedureStats.sql  #SQL
  
SELECT 
	 p.name AS [SP Name] ,  
	 qs.total_logical_reads AS [TotalLogicalReads] ,   
	 CASE WHEN qs.total_logical_reads  > 0 THEN	 qs.total_logical_reads / qs.execution_count ELSE 0
		  END	 AS [AvgLogicalReads] ,  
	 qs.execution_count ,   
	 CASE WHEN qs.total_elapsed_time > 0  THEN total_elapsed_time /1000 ELSE 0 END AS total_elapsed_timeMS ,  
	 CASE WHEN qs.execution_count > 0  THEN (total_elapsed_time /1000)/qs.execution_count ELSE 0 END AS [avg_elapsed_timeMS] ,  
	 qs.cached_time,  
	 db_name() AS DbName, 
	 @@ServerName AS ServerName   
FROM   
	sys.procedures AS p   
INNER JOIN   
	sys.dm_exec_procedure_stats AS qs ON p.[object_id] = qs.[object_id]  AND db_id() = database_id
ORDER BY   
	 qs.total_logical_reads DESC ;

> **DBA Space Calculations**
> 
> - Find details of procedure Server space (Query 5)
> - Database sizes and free space / total data and log space free on server

In [ ]:
-- DBA_DBSpace.sql  #SQL


USE [master]
GO
-----------------------------------------------------------------  
-- Object:   sp_DBA_DBSpaceCalc  
-- Written By:  Martin Croft  
-- Date Written:   
-- Purpose:   Returns space information   
-- Template:  1.0   
-- Usage:     
--      
-- Calls:   N/A  
-----------------------------------------------------------------  
-- Modified By  Modified Date  Description  Version  
-----------------------------------------------------------------  
-- {developer}  {date}    {description} {Version}  
-- Added 2 decimal placing rounding 
-----------------------------------------------------------------  

SET NOCOUNT ON

IF (SELECT object_id('Tempdb..#LogFiles')) IS NOT NULL DROP TABLE #LogFiles
CREATE TABLE #LogFiles
(
 DBName VARCHAR(100)
,LogSize DECIMAL(12,6)
,LogSpaceuse DECIMAL(12,6)
,LogSpaceFree DECIMAL(12,6) DEFAULT (0)
)

INSERT INTO #LogFiles
EXEC ('DBCC sqlperf(logspace)')

UPDATE #logfiles
SET LogSpaceFree =(LogSize /100) * LogSpaceuse

IF (SELECT object_id('Tempdb..#FileStats')) IS NOT NULL DROP TABLE #FileStats
CREATE TABLE #FileStats 

(
	ID INT IDENTITY(1,1)
	,FileID INT
	,FileGroup INT
	,TotalExtents INT
	,UsedExtents INT
	,Name sysname
	,FileName VARCHAR(200)
	,DBNAME sysname NULL
	,Updated INT DEFAULT (0)
	,srvName VARCHAR(30)
)


IF (SELECT object_id('Tempdb..#Databases')) IS NOT NULL DROP TABLE #Databases
CREATE TABLE #Databases
(
	 ID INT IDENTITY(1,1)
	,DBID INT
	,Name VARCHAR(200)
)

 --Get all database that are online and not snapshots
INSERT INTO #Databases (DBID,Name)
SELECT 
	database_id,
	Name 
FROM 
	master.sys.databases
WHERE 
		state_desc='ONLINE' 
	AND 
		source_database_id IS NULL


DECLARE @MaxID INT, @Loop INT

DECLARE @ExecStr VARCHAR(200), @Name VARCHAR(100),@Rowcount INT

SELECT @MaxID =MAX(ID) FROM #Databases
SET @Loop=1

WHILE @Loop <= @MaxID 
BEGIN
	SELECT @Name = Name from #databases where ID =@loop
	SELECT @ExecStr='USE ['+@Name+']; DBCC showfilestats' 

--Insert file stats into temp table for each DB

	INSERT INTO #FileStats 
	(	FileID
		,FileGroup
		,TotalExtents
		,UsedExtents
		,Name
		,FileName
	)
	
	EXEC (@ExecStr)
	SET @rowcount =@@rowcount 
	

	--SELECT @Loop , (@loop+@rowcount)

	UPDATE #FileStats
	SET DBNAME = @Name
		,Updated =1
		,srvName =@@servername
	WHERE 
		updated =0

	SET @loop=@loop+1
END


--Database Sizes 

SELECT 
	 @@servername AS SrvName
	,DBName
	,fileid
	,DATABASEPROPERTYEX( DBName, 'RECOVERY' ) AS Model
	,ROUND(cast(((TotalExtents * 64 *1.0 )/1024) as decimal(10,2)),-2) TotalSizeMg
	,cast((UsedExtents * 64 *1.0) /1024 as decimal (10,2)) UsedSizeMg
	,cast(ROUND(((TotalExtents * 64 *1.0 )/1024 )-((UsedExtents * 64 *1.0) /1024 ),2,-2)as decimal(10,2)) FreeMg
FROM 
	#filestats
ORDER BY 
	DBNAME,fileid
	---ROUND(((TotalExtents * 64 *1.0 )/1024 )-((UsedExtents * 64 *1.0) /1024 ),2,-2)  DESC

 --Data than can be reclaimed back
SELECT 
	SrvName
	,ROUND(CAST(SUM(ROUND(((TotalExtents * 64 *1.0 )/1024 )-((UsedExtents * 64 *1.0) /1024 ),2,-2))AS VARCHAR(50)),-2) [FreeMg - Data Size that can be claimed back]
FROM 
	#filestats
GROUP BY
	SrvName



 --Data than can be reclaimed back
SELECT 
	SrvName
	,SUM(cast((UsedExtents * 64 *1.0) /1024 as decimal (10,2))) [TotalUsedSizeMg]
FROM 
	#filestats
GROUP BY
	SrvName



 
--Dabtabase log sizes
SELECT
	 DBName
	,ROUND(CAST(LogSize AS VARCHAR(50)),-2) [LogSize Mg]
	,ROUND(CAST(LogSpaceuse AS VARCHAR(50)),-2)[LogSpaceused %]
	,ROUND(CAST((LogSize-LogSpaceFree) AS VARCHAR(50)),-2)[LogSpaceFree Mg]
FROM 
	#logfiles
ORDER BY 
	LogSpaceFree DESC




--Log space that can be reclaimed  

SELECT 
	ROUND(CAST(SUM(LogSize-LogSpaceFree)AS VARCHAR(MAX)),2) [Log Space that can be reclaimed] 
FROM 
	#logfiles 
